In [1]:
# import all libraries that are needed

import requests                 # requests on websites
from bs4 import BeautifulSoup   # html parsing

import pandas as pd             # pandas for data frame
import numpy as np              # numpy for array

import math                     # for math methods
import time                     # for sleep timer


from sql_functions import *     # functions from file for upload on schema
import psycopg2                 # for upload on engine

# Import the get_engine function from our sql_functions.
from sql_functions import get_engine #adjust this as necessary to match your sql_functions.py connection methods

# Import get_dataframe from sql_functions.py
from sql_functions import get_dataframe

# create a variable called engine using the get_engine function
engine = get_engine()

import datetime                 # for date and time
import datetime as dt           # for the csv file with the current date and time
from datetime import date       # for the current date

import re                       # for regular expressions

# https://jsonformatter.org/json-viewer
import json                     # for json files

import random                   # for random numbers

# https://jwalton.info/Matplotlib-custom-boxplots/
import itertools

import matplotlib.pyplot as plt # for plotting
# Will allow us to embed images in the notebook
%matplotlib inline
from matplotlib.cbook import _reshape_2D

# Seaborn visualization library
# https://towardsdatascience.com/visualizing-data-with-pair-plots-in-python-f228cf529166
import seaborn as sns

In [2]:
# create an empty dataframe
df_full = pd.DataFrame()
df_object = pd.DataFrame()
df_search = pd.DataFrame()
df_detail = pd.DataFrame()
df = pd.DataFrame()

In [3]:
# set up the link to the website
weblink = 'https://www.theblueground.com/furnished-apartments-london-uk?currency=GBP&language=en&offset='
print(weblink)

# set up the first page to scrape
pagesite = 0 # the website will start with 0 / # we set it to 10 to test the code

# create an empty list to store the blank slates
blank_slates_lst = [] 

# set the stop condition
stop_loop = "We’re sorry! We can’t seem to find any apartments that match your search." 

https://www.theblueground.com/furnished-apartments-london-uk?currency=GBP&language=en&offset=


In [4]:
# check the link
full_link = weblink + str(pagesite)
print(full_link)


https://www.theblueground.com/furnished-apartments-london-uk?currency=GBP&language=en&offset=0


In [5]:
# parse the html and save it into a BeautifulSoup instance
#page = requests.get(weblink +  f'offset={ pagesite }&items=18')
page = requests.get(full_link)
# parse the html and save it into a BeautifulSoup instance
html = page.content
bs = BeautifulSoup(html, 'html.parser')

In [6]:
print(weblink +   f'{ pagesite }' )

https://www.theblueground.com/furnished-apartments-london-uk?currency=GBP&language=en&offset=0


In [7]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html

def get_blueground_id(df_search):
    # create an empty list to store the blueground_id
    blueground_id_lst = []
    # get the blueground_id from the dataframe and the column get_url_to_detail_page
    for blueground_id in df_search.get_url_to_detail_page: # we get the url from the dataframe
        blueground_id_lst.append(blueground_id.rsplit('-', 1)[-1]) # we get the last part of the url
    return blueground_id_lst

In [8]:
def get_neighborhoods(bs):
    neighborhood_names = bs.find_all("div", {"class":"name-place"})
    neighborhood_names_lst = (neighborhood_name.get_text() for neighborhood_name in neighborhood_names)
    neighborhood_names_lst = [neighborhood_name.strip() for neighborhood_name in neighborhood_names_lst]
    neighborhood_names_lst = [i.rsplit(',', 1)[-1] for i in neighborhood_names_lst]
    neighborhood_names_lst = [i.rsplit('-', 1)[0] for i in neighborhood_names_lst]
    neighborhood_names_lst = [i.strip() for i in neighborhood_names_lst]
    return neighborhood_names_lst

In [9]:
def get_property_type(bs):
    # get the property_type out of the apartment amenities
    lst_name = []
    property_typs = bs.find_all(class_="main-amenities")
    property_typs_lst = [property_type.get_text() for property_type in property_typs]
    property_typs_lst = [property_type.strip() for property_type in property_typs_lst]
    property_typs_lst = [i.split('o', 1)[0] for i in property_typs_lst]
    property_typs_lst = [i.replace('Bedr', 'Bedroom') for i in property_typs_lst]
    property_typs_lst = [i.replace('Studi', 'Studio') for i in property_typs_lst]
    property_typs_lst = [i.rsplit(' ', 1)[-1] for i in property_typs_lst]
    return property_typs_lst

In [10]:
def get_bedrooms(bs):
    # get the main-amenities of the apartments
    lst_name = []
    bedrooms = bs.find_all(class_="main-amenities")
    bedrooms_lst = [bedroom.get_text() for bedroom in bedrooms]
    bedrooms_lst = [bedroom.strip() for bedroom in bedrooms_lst]
    bedrooms_lst = [i.split('o', 1)[0] for i in bedrooms_lst]
    bedrooms_lst = [i.replace('Bedr', 'Bedroom') for i in bedrooms_lst]
    bedrooms_lst = [i.replace('Studi', 'Studio') for i in bedrooms_lst]
    return bedrooms_lst

In [11]:
def get_bathroom(bs):
    # get the main-amenities of the apartments
    lst_name = []
    bathrooms= bs.find_all(class_="main-amenities")
    bathrooms_lst = [bathroom.get_text() for bathroom in bathrooms]
    bathrooms_lst = [bathroom.strip() for bathroom in bathrooms_lst]
    bathrooms_lst = [i.rsplit('o', 1)[-1] for i in bathrooms_lst]
    bathrooms_lst = [i.replace('m', '') for i in bathrooms_lst]

    return bathrooms_lst

In [12]:
def get_price_pcm(bs):
    # get the price per month
    lst_name = []
    prices = bs.find_all(class_= "price__amount")
    for price in prices:
        lst_name.append(
            price.get_text()
                .strip()
                .replace(',', '')
        )
    return lst_name

In [13]:
def get_object_title(bs):
    # get the names of all the apartments
    lst_name = []
    object_titles = bs.find_all(class_="listing-name")
    for object_title in object_titles:
        lst_name.append(
            object_title.get_text()
                .strip()
        )
    return lst_name


In [14]:
def get_availability(bs):
    # get the availability of the apartments
    lst_availability = []
    availability = bs.find_all(class_="availability__date")
    for avail in availability:
        lst_availability.append(
            avail.get_text()
                .strip()
        )
    return lst_availability


In [15]:
def get_url_to_detail_page(bs, maximus):
    url_lst = []
    count = 0
    while count <= int(maximus):
        
        # this will get us the link to the detail page
        class_with_link = bs.find_all(class_="ui-image-carousel")
        # with the [] we can select the elment we want to get
        for a in class_with_link[count].find_all('a', href=True):
            url_lst.append(a['href'])
        count += 1
    #print(url_lst)
    return url_lst

In [16]:
today = date.today()
print("Today's date:", today)

Today's date: 2022-10-21


In [17]:
# https://flexiple.com/python/check-if-list-is-empty-python/
# Solution 3: Using len() function
# The len() function returns the number of items in a list. If the list is empty, it returns 0.

In [19]:
# https://flexiple.com/python/check-if-list-is-empty-python/
# Solution 3: Using len() function
# The len() function returns the number of items in a list. If the list is empty, it returns 0.
while len(blank_slates_lst) == 0: # start and endpoint of the for-loop
    # pause the loop for a raondom time to reduce the load on the server
    time.sleep(random.randint(5,15)/random.randint(1,9))

    # ----------------- get the data from the website -----------------
    # get the content of the website
    page = requests.get(weblink +  f'{ pagesite }')
    print(weblink +   f'{ pagesite }' )
    # parse the html and save it into a BeautifulSoup instance
    html = page.content
    bs = BeautifulSoup(html, 'html.parser')

    # ----------------- insert the data into the dataframe -----------------
    # create a pandas dataframe for the names and prices
    blueground_dict = {
        #'platform_id', --> we get this from the detail page
        'platform': 'Blueground',
        'neighbourhood': get_neighborhoods(bs),
        'property_type': get_property_type(bs),
        'bedrooms': get_bedrooms(bs),
        'bathroom': get_bathroom(bs),
        'price_pcm': get_price_pcm(bs),
        'title': get_object_title(bs),
        'furniture': 'furnished',
        'detailed_furniture': 'furnished',
        'available_from': get_availability(bs),
        #'size': , --> we get this from the detail page
        'title': get_object_title(bs),
        'scraping_date_main': date.today(), # today's date
        'available_today': ''
        }
    
    # ----------------- work with the datafram to get the URL and the ID -----------------
    # we now have a dataframe, we can use this to get a counter for the URL
    df_page = pd.DataFrame(blueground_dict)


    # ----------------- check if we reached the end of the pages -----------------
    blank_slates = bs.find_all(class_="blank-slate__criteria")
    blank_slates_lst = (blank_slate.get_text() for blank_slate in blank_slates)
    blank_slates_lst = [blank_slate.strip() for blank_slate in blank_slates_lst]

    # increase the pagesite by 1
    #pagesite += 1



https://www.theblueground.com/furnished-apartments-london-uk?currency=GBP&language=en&offset=0


ValueError: All arrays must be of the same length

In [22]:
page = requests.get(weblink +  f'{ pagesite }')
print(weblink +   f'{ pagesite }' )
# parse the html and save it into a BeautifulSoup instance
html = page.content
bs = BeautifulSoup(html, 'html.parser')

# ----------------- insert the data into the dataframe -----------------
# create a pandas dataframe for the names and prices
blueground_dict = {
    #'platform_id', --> we get this from the detail page
    'platform': 'Blueground'

    }

# ----------------- work with the datafram to get the URL and the ID -----------------
# we now have a dataframe, we can use this to get a counter for the URL
df_page = pd.DataFrame(blueground_dict, index=[0])

KeyboardInterrupt: 

In [ ]:
while len(blank_slates_lst) == 0: # start and endpoint of the for-loop
    # pause the loop for a raondom time to reduce the load on the server
    time.sleep(random.randint(5,15)/random.randint(1,9))

    # ----------------- insert the data into the dataframe -----------------
    # create a pandas dataframe for the names and prices
    blueground_dict = {
        #'platform_id', --> we get this from the detail page
        'platform': 'blueground',
        'neighbourhood': get_neighborhoods(bs),
        'property_type': get_property_type(bs),
        'bedrooms': get_bedrooms(bs),
        'bathroom': get_bathroom(bs),
        'price_pcm': get_price_pcm(bs),
        'title': get_object_title(bs),
        'furnished': 'furnished',	
        'detailed_furniture': 'furnished',
        'available_from': get_availability(bs),
        #'size': , --> we get this from the detail page
        'title': get_object_title(bs),
        'scraping_date_main': today, # today's date
        'available_today': ''        
        }
    
    # ----------------- work with the datafram to get the URL and the ID -----------------
    # we now have a dataframe, we can use this to get a counter for the URL
    df_page = pd.DataFrame(blueground_dict)
    print(df_page)

    # we create a variable to store the number of rows in the dataframe
    maximus = df_page.index.max() # we give it the max value of the index

    # we create a list to store the urls
    # we check if we reached the end of the existing df, to stop and not get an error
    if np.isnan(maximus):
        break
    else:
        df_page['get_url_to_detail_page'] = pd.Series(get_url_to_detail_page(bs, maximus))
    # we can now add the dataframe to the full dataframe
    df_search = pd.concat([df_search, df_page], axis=0, ignore_index=True)


    # ----------------- check if we reached the end of the pages -----------------
    blank_slates = bs.find_all(class_="blank-slate__criteria")
    blank_slates_lst = (blank_slate.get_text() for blank_slate in blank_slates)
    blank_slates_lst = [blank_slate.strip() for blank_slate in blank_slates_lst]

    # increase the pagesite by 1
    pagesite += 1


# ----------------- get the ID from URL -----------------
# we want the ID on the first place, so we can use it to merge the dataframes, if needed
# Third position would be at index 2, because of zero-indexing.
df_search.insert(0, 'platform_id', get_blueground_id(df_search))